In [229]:

import datetime
import pandas as pd
import yfinance as yf
import pandas_datareader.data as web
import numpy as np
import scipy as sp
from scipy.optimize import minimize



In [230]:
# FANG

jm = yf.download('JPM', start="2022-05-01", end="2023-05-01")
ap = yf.download('AAPL', start="2022-05-01", end="2023-05-01")
am = yf.download('AMZN', start="2022-05-01", end="2023-05-01")
wm = yf.download('WMT', start="2022-05-01", end="2023-05-01")
tg = yf.download('TGT', start="2022-05-01", end="2023-05-01")
ms = yf.download('MSFT', start="2022-05-01", end="2023-05-01")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [231]:
stocks = pd.concat([jm['Adj Close'], ap['Adj Close'], am['Adj Close'], wm['Adj Close'], tg['Adj Close'], ms['Adj Close']],axis=1)




In [232]:
stocks.columns = ['JP Morgan', 'Apple', 'Amazon', 'Walmart', 'Target', 'Microsoft']
stocks.tail()
#exporting csv to csv for database
stocks.to_csv('df2db.csv', index=False)

In [233]:
## Change in returns over time

returns = stocks/stocks.shift(1)
returns

,JP Morgan,Apple,Amazon,Walmart,Target,Microsoft
Date,,,,,,
2022-05-02,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-03,1.021420,1.009622,0.998020,1.003487,1.012011,0.990544
2022-05-04,1.033081,1.041008,1.013480,1.013835,1.021029,1.029101
2022-05-05,0.974980,0.944284,0.924390,0.991432,0.971908,0.956445
2022-05-06,0.998386,1.004727,0.985959,0.979180,0.975949,0.990554
...,...,...,...,...,...,...
2023-04-24,1.001352,1.001879,0.992988,1.006788,1.010852,0.986037
2023-04-25,0.978256,0.990564,0.965728,0.992210,0.961388,0.977464
2023-04-26,0.982276,0.999939,1.023496,0.995646,0.986295,1.072435


In [234]:
## Find mean of returns

returns.mean()

JP Morgan    1.000847
Apple        1.000539
Amazon       0.999749
Walmart      1.000171
Target       0.998998
Microsoft    1.000582
dtype: float64

In [235]:
logReturns = np.log(returns)
logReturns

,JP Morgan,Apple,Amazon,Walmart,Target,Microsoft
Date,,,,,,
2022-05-02,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-03,0.021194,0.009576,-0.001982,0.003481,0.011940,-0.009501
2022-05-04,0.032546,0.040190,0.013390,0.013740,0.020811,0.028685
2022-05-05,-0.025338,-0.057329,-0.078622,-0.008605,-0.028495,-0.044532
2022-05-06,-0.001615,0.004716,-0.014141,-0.021039,-0.024345,-0.009491
...,...,...,...,...,...,...
2023-04-24,0.001351,0.001877,-0.007037,0.006765,0.010794,-0.014061
2023-04-25,-0.021984,-0.009480,-0.034873,-0.007820,-0.039377,-0.022794
2023-04-26,-0.017883,-0.000061,0.023224,-0.004364,-0.013800,0.069932


In [236]:

pbar = logReturns.mean() * 252
sigma = logReturns.cov() *252

In [237]:
rMin = 0.02
def riskFunction(w):
    return np.dot(w.T,np.dot(sigma,w))

w0 = [0.166, 0.166, 0.166, 0.166, 0.166, 0.167]
bounds = ((0,1), (0,1), (0,1), (0,1), (0,1), (0,1))

def checkMinimumRertun(w):
    RHS = rMin - np.sum(pbar * w)
    return RHS
def checkSumToOne(w):
    return np.sum(w) - 1 

constraints = ({'type': 'eq', 'fun': checkMinimumRertun},{'type': 'eq', 'fun': checkSumToOne})
w_opt = minimize(riskFunction, w0, method='SLSQP', bounds=bounds, constraints=constraints)

In [238]:
w_opt

     fun: 0.05115837686470404
     jac: array([0.08309396, 0.0949369 , 0.12690627, 0.10352511, 0.15076456,
       0.09321559])
 message: 'Optimization terminated successfully'
    nfev: 71
     nit: 10
    njev: 10
  status: 0
 success: True
       x: array([0.25381753, 0.01236147, 0.04539653, 0.54974009, 0.07737082,
       0.06131356])

In [239]:
w_scipy = w_opt.x
w_scipy

array([0.25381753, 0.01236147, 0.04539653, 0.54974009, 0.07737082,
       0.06131356])

In [240]:
risk_scipy = riskFunction(w_scipy)
risk_scipy

0.05115837686470404

In [241]:
np.sum(pbar*w_scipy)


0.019999999999000418

In [242]:
def markowitz(rMin,sigma,pbar):
    N = len(sigma)
    o = np.ones(N)
    sigmaInv = np.linalg.inv(sigma)
    a = np.dot(pbar.T, np.dot(sigmaInv,pbar))
    b = np.dot(pbar.T,np.dot(sigmaInv,o))
    c = np.dot(o.T,np.dot(sigmaInv,o))
    return (1/(a*c - b**2)) * np.dot(sigmaInv,((c*rMin - b)*pbar + (a-b*rMin)*o))

In [243]:
analysis = markowitz(rMin,sigma,pbar)
analysis

array([0.2514959 , 0.00861659, 0.04222783, 0.55143767, 0.07852841,
       0.06769358])